In [12]:
import pandas as pd
import pandas as pd
import numpy as np
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
torch.manual_seed(42)

df = pd.read_csv('/home/csgrad/kaushik3/LLM/Kaggle_63LLMs/FLAN/human_flan.csv')

In [13]:
from transformers import RobertaModel, RobertaTokenizer

# Load the model
model_name = "roberta-large"
model = RobertaModel.from_pretrained(model_name)

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
df['source'].value_counts()

Human            278018
Flan-T5-Base       7358
Flan-T5-Large      7352
Flan-T5-Small      7320
Flan-T5-XXL        7305
Flan-T5-XL         7243
Name: source, dtype: int64

In [3]:
AI = df[df['source'].str.startswith('Flan')]
AI.head()


,text,source,prompt_id,text_length,word_count,label_map
23,"Apple announced updates to iOS, iPadOS, MacOS,...",Flan-T5-Base,0,220,38,1
24,"Then, select a bedtime that allows your child ...",Flan-T5-Large,0,211,40,1
46,Overall...a sensational effort. From the atmos...,Flan-T5-XL,0,248,36,1
51,Yes and no. Are they going to have to share th...,Flan-T5-XXL,0,174,33,1
55,Here's an obscure one. List of US cities with ...,Flan-T5-XL,0,197,36,1


In [4]:
Human = df[df['source'] == 'Human']
Human = Human[:len(AI)]

In [5]:
print(len(Human))
print(len(AI))

36578
36578


In [11]:
def generate_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        #embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
        last_hidden_states = outputs.last_hidden_state
        embeddings = last_hidden_states[:,0,:]
    return embeddings.numpy()
import numpy as np
def remove_outliers(embeddings):
        # Calculate the mode of the embeddings
        mode_val = np.mean(embeddings[0], axis=None)
        
        # Replace outliers with the mode value
        embeddings[np.abs(embeddings - mode_val) > 2 * np.std(embeddings)] = mode_val
        
        return embeddings
def min_max_normalize(embeddings):
    # Find the minimum and maximum values in the embeddings
    min_val = np.min(embeddings)
    max_val = np.max(embeddings)
    
    # Normalize the embeddings to range [0, 255]
    normalized_embeddings = 255 * (embeddings - min_val) / (max_val - min_val)
    
    return normalized_embeddings.astype(np.uint8)  # Convert to uint8 for integer values between 0 and 255

def reshape_embeddings(embeddings):
    # Reshape the embeddings to 3D array
    return embeddings.reshape(32,32)

In [9]:
AI['embeddings'] = AI['text'].apply(generate_bert_embeddings)
AI['embeddings1'] = AI['embeddings'].apply(remove_outliers)
AI['normalized_embeddings'] = AI['embeddings1'].apply(min_max_normalize)
AI['reshaped_embeddings'] = AI['normalized_embeddings'].apply(reshape_embeddings)

In [10]:
AI_embeddings = AI['embeddings']

In [11]:

with open('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/embeddings/flan_embeddings.npy', 'wb') as f:
    #for i in range(len(gpt['embeddings'])):
    np.save(f, AI_embeddings)
    

In [4]:
import numpy as np

AI_embeddings = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/embeddings/flan_embeddings.npy', allow_pickle=True)


In [16]:
for i in range(len(AI_embeddings)):
    
    AI_embeddings[i] = remove_outliers(AI_embeddings[i])
    AI_embeddings[i] = min_max_normalize(AI_embeddings[i])
    AI_embeddings[i] = reshape_embeddings(AI_embeddings[i])


In [7]:
len(AI_embeddings)
AI_embeddings.shape

(36578,)

In [17]:
import matplotlib.pyplot as plt

for i in range(23475, len(AI_embeddings)):
    plt.imshow(AI_embeddings[i], cmap='gray')  # Assuming grayscale image
    plt.axis('off')  # Hide axes
    plt.savefig('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/HumanvsLLM/Flan/'+str(i)+'.png', bbox_inches='tight', pad_inches=0)
    plt.close()


In [18]:
import os

folder_path = '/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/HumanvsLLM/Flan/'  # Replace with the actual folder path

file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

print("Number of files in the folder:", file_count)


Number of files in the folder: 36578
